In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# pip install -U scikit-learn

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import numpy as np
import pandas as pd

In [5]:
!pip install fasttext
import fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=4395599 sha256=a3822eba50136cc9e60c31936b526ccdb4a431d4f7689195d2976d479057a9da
  Stored in directory: /root/.cache/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


In [6]:
df = pd.read_csv('drive/MyDrive/TextAnalytics/preprocessed.csv',engine="python", encoding="utf-8", error_bad_lines=False)
df.sample(5)

<ipython-input-6-7fde084fdf7a>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('drive/MyDrive/TextAnalytics/preprocessed.csv',engine="python", encoding="utf-8", error_bad_lines=False)


,text,class,char_length,token_length
52714,life horrible getting worse losing dirt poor f...,0,107,20
83435,honestly tho hate,0,17,3
96561,woo hoo hooray know care probably sound awful ...,0,167,25
86419,day asking people uthecollegedropout today,0,50,6
217699,song rate genre prefer sad box indie shoe gaze...,0,96,16


In [7]:
model = fasttext.load_model('drive/MyDrive/TextAnalytics/cc.en.300.bin')
def get_text_vector(sentence): return model.get_sentence_vector(sentence)

In [8]:
df['text_vectors'] = df['text'].apply(get_text_vector)
df.sample(5)

,text,class,char_length,token_length,text_vectors
169660,think need help 've voice head telling dark th...,0,99,18,"[-0.017585093, 0.016359214, 0.034307793, 0.080..."
195527,teen masturbate old,0,20,3,"[0.034817718, -0.003082022, -0.03943831, 0.056..."
69250,advice family suicide attempt survivor sure ri...,1,557,80,"[-0.012809544, 0.0143346945, 0.00796114, 0.065..."
161511,worth trying anymore honestly know anymore fee...,1,493,78,"[-0.0032276418, 0.010443854, -0.001969015, 0.0..."
133792,anymore struggling long ant cut anymore forced...,1,309,49,"[-0.0048890184, 0.0021893978, 0.010343137, 0.0..."


In [9]:
X, y = df['text_vectors'], df['class']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# **Model with default hyperparamers**

In [11]:
logreg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, max_iter=10000)
logreg_model = logreg.fit(list(X_train),y_train)

y_pred = logreg_model.predict(list(X_test))

#Confusion Matrix
clf_tn, clf_fp, clf_fn, clf_tp = confusion_matrix(y_test, y_pred).ravel()
#measurements
clf_accuracy = (clf_tp+clf_tn)/(clf_tp+clf_tn+clf_fp+clf_fn)
print(" Accuracy", round(clf_accuracy,4))
clf_precision = clf_tp/(clf_tp+clf_fp)
clf_recall = clf_tp/(clf_tp+clf_fn)
clf_f1_score = (2*clf_precision)*(clf_recall)/(clf_precision+clf_recall)
print(" Precision",round(clf_precision,4),"\n","Recall",round(clf_recall,4),"\n","F1",round(clf_f1_score,4))

[LibLinear] Accuracy 0.892
 Precision 0.8781 
 Recall 0.9118 
 F1 0.8946


# **Model with Hyperparameter tuning**

In [ ]:
param_grid_ = {'penalty' : ['l1', 'l2'],
              'C': np.logspace(-4, 4, 20)}
logreg_model_search = GridSearchCV(LogisticRegression(verbose=1, solver='liblinear',random_state=0, max_iter=1000), 
                                   cv=5, param_grid=param_grid_, )
logreg_model_search.fit(list(X_train), y_train)

[LibLinear]

In [ ]:
print(logreg_model_search.best_params_)
print(logreg_model_search.best_score_)

In [ ]:
logreg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, max_iter=1000, C=10000, penalty='l2')
logreg_model = logreg.fit(list(X_train),y_train)

y_pred = logreg_model.predict(list(X_test))

#Confusion Matrix
clf_tn, clf_fp, clf_fn, clf_tp = confusion_matrix(y_test, y_pred).ravel()
#measurements
clf_accuracy = (clf_tp+clf_tn)/(clf_tp+clf_tn+clf_fp+clf_fn)
print(" Accuracy", round(clf_accuracy,4))
clf_precision = clf_tp/(clf_tp+clf_fp)
clf_recall = clf_tp/(clf_tp+clf_fn)
clf_f1_score = (2*clf_precision)*(clf_recall)/(clf_precision+clf_recall)
print(" Precision",round(clf_precision,4),"\n","Recall",round(clf_recall,4),"\n","F1",round(clf_f1_score,4))